In [ ]:
import numpy as np
from src.perceptron import Perceptron
from src.trainer import TrainerConfig, train_perceptron, evaluate_perceptron
from src.readers import read_csv
from src.plotter import plot_arrays, plot_bars
from src.theta_funcs import ThetaFunction
from src.error_funcs import cost_average

In [ ]:
dataset, dataset_outputs = read_csv('TP3-ej2-conjunto.csv')

config = TrainerConfig.from_file("ejercicio2-lineal-config.json")

initial_weights = np.random.random(len(dataset[0]) + 1) * 2 - 1

In [ ]:
perceptron_lin = Perceptron(
    initial_weights=np.copy(initial_weights),
    theta_func=config.theta
)

print(f"Initial weights for LINEAR perceptron: {perceptron_lin.w}")

error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_lin = train_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_lin,
    dataset=dataset,
    dataset_outputs=dataset_outputs,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LINEAR perceptron after training for {result_lin.epoch_num} epoch{'' if result_lin.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_lin.end_reason.value}")
print(f"Final weights: {perceptron_lin.w}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-tanh-config.json")

perceptron_tanh = Perceptron(
    initial_weights=np.copy(initial_weights),
    theta_func=config.theta
)
outputs_tanh = config.scaler.scale(dataset_outputs)

print(f"Initial weights for TANH perceptron: {perceptron_tanh.w}")

error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_tanh = train_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_tanh,
    dataset=dataset,
    dataset_outputs=outputs_tanh,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"TANH perceptron after training for {result_tanh.epoch_num} epoch{'' if result_tanh.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_tanh.end_reason.value}")
print(f"Final weights: {perceptron_tanh.w}")

print(f"Error obtained by inverse-scaling outputs: {cost_average(dataset_outputs, config.scaler.reverse([perceptron_tanh.evaluate(dat) for dat in dataset]))}")

In [ ]:
config = TrainerConfig.from_file("ejercicio2-logistic-config.json")

perceptron_logis = Perceptron(
    initial_weights=np.copy(initial_weights),
    theta_func=config.theta
)
outputs_logis = config.scaler.scale(dataset_outputs)

print(f"Initial weights for LOGISTIC perceptron: {perceptron_logis.w}")

error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron initially has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")

In [ ]:
result_logis = train_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    config=config
)

In [ ]:
error = evaluate_perceptron(
    perceptron=perceptron_logis,
    dataset=dataset,
    dataset_outputs=outputs_logis,
    error_func=config.error_func,
    print_output=True,
    acceptable_error=config.acceptable_error
)

print(f"LOGISTIC perceptron after training for {result_logis.epoch_num} epoch{'' if result_logis.epoch_num == 1 else 's'} has an error of {error} {'✅' if error<=config.acceptable_error else '❌'}")
print(f"Finished because: {result_logis.end_reason.value}")
print(f"Final weights: {perceptron_logis.w}")

print(f"Error obtained by inverse-scaling outputs: {cost_average(dataset_outputs, config.scaler.reverse([perceptron_logis.evaluate(dat) for dat in dataset]))}")

In [ ]:
def map_to_percentage(array, a, b):
    range_length = b - a
    return [(value - a) / range_length for value in array]

lin_range = (0, 3872)
tanh_range = (0, 2.0)
logis_range = (0.0, 0.5)

erros_lin_percentage = map_to_percentage(result_lin.error_history, *lin_range)
erros_tanh_percentage = map_to_percentage(result_tanh.error_history, *tanh_range)
erros_logis_percentage = map_to_percentage(result_logis.error_history, *logis_range)
errors = [erros_lin_percentage, erros_tanh_percentage, erros_logis_percentage]
errors_labels = ["LINEAR", "TANH", "LOGISTIC"]

plot_arrays(errors, errors_labels, "Average Percentual Error per Epoch", "Epoch", "Error").plot()

In [ ]:
from src.dataset_selectors import split_random_by_percentage, handpick_diverse_training_dataset
from src.runner import run_n_times

In [ ]:
lin_range = (0, 3872)
tanh_range = (0, 1.0)
logis_range = (0.0, 1.0)

def split_random_by_percentage_and_theta(percentage: float, theta: ThetaFunction, theta_name: str):
    train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs = split_random_by_percentage(dataset, dataset_outputs, percentage)

    if(config.theta != "lineal"):
        train_dataset_outputs = config.scaler.scale(train_dataset_outputs)
        test_dataset_outputs = config.scaler.scale(test_dataset_outputs)
    name = f"{percentage*100}% Training {100-percentage*100}% Test para theta {theta_name}"
    
    return train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs, name

def split_handpicked_by_theta(theta: ThetaFunction, theta_name: str):
    train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs = handpick_diverse_training_dataset(dataset, dataset_outputs)

    if(config.theta != "lineal"):
        train_dataset_outputs = config.scaler.scale(train_dataset_outputs)
        test_dataset_outputs = config.scaler.scale(test_dataset_outputs)
        name = f"Handpick para theta {theta_name}"
    
    return train_dataset, train_dataset_outputs, test_dataset, test_dataset_outputs, name
    

config_linear = TrainerConfig.from_file("ejercicio2-lineal-config.json")
config_tanh = TrainerConfig.from_file("ejercicio2-tanh-config.json")
config_logistic = TrainerConfig.from_file("ejercicio2-logistic-config.json")
configs = [config_linear, config_tanh, config_logistic]
configs_names = ["LINEAR", "TANH", "LOGISTIC"]
percentages = [0.1, 0.5, 0.9]

training_errors = []
test_errors = []

for config in configs:
    config_name = configs_names[configs.index(config)]
    (
        training_error_mean,
        training_error_stdev,
        test_error_mean,
        test_error_stdev,
        name
    ) = run_n_times(
        10,
        lambda: split_handpicked_by_theta(config.theta, config_name),
        config,
    )

    range = lin_range if config_name == "LINEAR" else tanh_range if config_name == "TANH" else logis_range

    training_error = {
        "name": f"{config_name} - Handpick",
        "mean": map_to_percentage([training_error_mean], *range)[0],
        "stdev": map_to_percentage([training_error_stdev], *range)[0]
    }

    training_errors.append(training_error)

    test_error = {
        "name": f"{config_name} - Handpick",
        "mean": map_to_percentage([test_error_mean], *range)[0],
        "stdev": map_to_percentage([test_error_stdev], *range)[0]
    }

    test_errors.append(test_error)

    for percentage in percentages:
        (
            training_error_mean,
            training_error_stdev,
            test_error_mean,
            test_error_stdev,
            name
        ) = run_n_times(
            10,
            lambda: split_random_by_percentage_and_theta(percentage, config.theta, config_name),
            config,
        )

        training_error = {
            "name": f"{config_name} - {percentage*100}%",
            "mean": map_to_percentage([training_error_mean], *range)[0],
            "stdev": map_to_percentage([training_error_stdev], *range)[0]
        }

        training_errors.append(training_error)

        test_error = {
            "name": f"{config_name} - {percentage*100}%",
            "mean": map_to_percentage([test_error_mean], *range)[0],
            "stdev": map_to_percentage([test_error_stdev], *range)[0]
        }

        test_errors.append(test_error)

plot_bars(
    [training_error["mean"] for training_error in training_errors],
    [training_error["stdev"] for training_error in training_errors],
    [training_error["name"] for training_error in training_errors],
    "Error Promedio de Entrenamiento",
    "Dataset",
    "Error"
).plot()

plot_bars(
    [test_error["mean"] for test_error in test_errors],
    [test_error["stdev"] for test_error in test_errors],
    [test_error["name"] for test_error in test_errors],
    "Error Promedio de Test",
    "Dataset",
    "Error"
).plot()



In [ ]:
# Plot only TANH 90% and LOGISTIC 90% for training
test_error_linear90 = [train_error for train_error in training_errors if train_error["name"].startswith('LINEAR - 90')][0]
test_error_tanh90 = [train_error for train_error in training_errors if train_error["name"].startswith('TANH - 90')][0]
test_error_log90 = [train_error for train_error in training_errors if train_error["name"].startswith('LOGISTIC - 90')][0]

plot_bars(
    [test_error_linear90["mean"], test_error_tanh90["mean"], test_error_log90["mean"]],
    [test_error_linear90["stdev"], test_error_tanh90["stdev"], test_error_log90["stdev"]],
    [test_error_linear90["name"], test_error_tanh90["name"], test_error_log90["name"]],
    "Error Promedio de Entrenamiento",
    "Dataset",
    "Error"
).plot()

In [ ]:
# Plot training only for each config_name
for config_name in configs_names:
    plot_bars(
        [training_error["mean"] for training_error in training_errors if training_error["name"].startswith(config_name)],
        [training_error["stdev"] for training_error in training_errors if training_error["name"].startswith(config_name)],
        [training_error["name"] for training_error in training_errors if training_error["name"].startswith(config_name)],
        "Error Promedio de Entrenamiento",
        "Dataset",
        "Error"
    ).plot()

In [ ]:
# Plot together TANH and LOGISTIC for test
plot_bars(
    [test_error["mean"] for test_error in test_errors if test_error["name"].startswith("TANH") or test_error["name"].startswith("LOGISTIC")],
    [test_error["stdev"] for test_error in test_errors if test_error["name"].startswith("TANH") or test_error["name"].startswith("LOGISTIC")],
    [test_error["name"] for test_error in test_errors if test_error["name"].startswith("TANH") or test_error["name"].startswith("LOGISTIC")],
    "Error Promedio de Test",
    "Dataset",
    "Error"
).plot()

In [ ]:
#Plot only the test for TANH or LOGISTIC
for config_name in configs_names:
    if config_name == "LINEAR":
        continue
    plot_bars(
        [test_error["mean"] for test_error in test_errors if test_error["name"].startswith(config_name)],
        [test_error["stdev"] for test_error in test_errors if test_error["name"].startswith(config_name)],
        [test_error["name"] for test_error in test_errors if test_error["name"].startswith(config_name)],
        "Error Promedio de Test",
        "Dataset",
        "Error"
    ).plot()

In [ ]:
# Plot only TANH 90% and LOGISTIC 90% for test
test_error_tan90 = [test_error for test_error in test_errors if test_error["name"].startswith('TANH - 90')][0]
test_error_log90 = [test_error for test_error in test_errors if test_error["name"].startswith('LOGISTIC - 90')][0]

plot_bars(
    [test_error_tan90["mean"], test_error_log90["mean"]],
    [test_error_tan90["stdev"], test_error_log90["stdev"]],
    [test_error_tan90["name"], test_error_log90["name"]],
    "Error Promedio de Test",
    "Dataset",
    "Error"
).plot()
